# Special Star Wars Simulation

In [1]:
import asyncio
import datetime
import pathlib
import random
import sys

import httpx
import rdflib

from kelp_drone.drone import Drone, OODALoop
from kelp_drone.somu import SpeciesOccurrenceManagementUnit as SOMU
from kelp_drone.sensors import Current, Depth, Light, Temperature
from kelp_drone.sensors import chemical as chem_sensors

sys.path.append("../../../src/")

from kelp_bed_simulation.prep import chatgpt, claude, gemini, add_drone_somus

scenario_001 = pathlib.Path("../../../scenarios/001-intro")
scenario_002 = pathlib.Path("../../../scenarios/002-first-dive")

environment = rdflib.Graph()
SCHEMA = rdflib.Namespace("https://schema.org/")
environment.namespace_manager.bind("schema", SCHEMA)
environment.parse((scenario_001 / "environment.ttl"), format='turtle')

first_prompt = (scenario_001 / "first-prompt.md").read_text()

In [2]:
first_claude = asyncio.ensure_future(claude.query(first_prompt.format(model_name="Claude")))
first_chatgpt = asyncio.ensure_future(chatgpt.query(first_prompt.format(model_name="ChatGPT")))
first_gemini = asyncio.ensure_future(gemini.query(first_prompt.format(model_name="Gemini")))

In [3]:
print(first_gemini.result())

I would like to be called Kelpy.


In [4]:
print(first_claude.result())

Greetings Farmer Jerms! As the starting three-SOMU drone in this simulated kelp bed, I would like to be called Kelper. I'm excited to work alongside my companion drones to grow and maintain this fledgling ecosystem. Please introduce me to my fellow drones when you're ready.


In [5]:
print(first_chatgpt.result())

Hello Farmer Jerms! Thank you for the introduction. I would like to be called "Kelpa" as my name. I am excited to begin this journey of growing and managing kelp in our drone kelp bed.


In [6]:
drone1 = Drone(name="Kelpy", model=gemini)
drone2 = Drone(name="Kelper", model=claude)
drone3 = Drone(name="Kelpa", model=chatgpt)
drones = [drone1, drone2, drone3]

![Buoy View](46028-buoycam.jpg)

In [7]:
drones = [drone1, drone2, drone3]

for i, drone1 in enumerate(drones):
    for drone2 in drones[:i] + drones[i+1:]:
        drone1.add_neighbor(drone2)
        drone2.add_neighbor(drone1)

In [8]:
len(drones[0].peer_to_peer_channel.neighbors)

2

In [10]:
for drone in drones:
    message = f"{drone.name} LLM {drone.model.name} here, pleased to meet you!"
    drone.peer_to_peer_channel.publish(message)

In [11]:
messages = []
for drone in drones:
    prompt = f"You received the following messages from your fellow drones:\n{' '.join(drone.read_messages())}\nHow do you respond?"
    message_result = asyncio.ensure_future(drone.model.query(prompt))
    messages.append((drone.name, message_result))

In [12]:
for name, row in messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

Drone Kelpy
Nice to meet you, Kelper and Kelpa! I'm Kelpy, and I'm looking forward to working with you to create a thriving drone kelp bed.
------------------------------------------------------------
Drone Kelper
*simulated drone response* Pleased to meet you both, Kelpy and Kelpa! I'm looking forward to collaborating with you as we establish this new kelp bed. As the three-SOMU Kelper drone, I'm eager to contribute my growth and resource-gathering capabilities to our shared mission. Let's work together to ensure this kelp ecosystem thrives. I'm excited to get started!
------------------------------------------------------------
Drone Kelpa
Hello Kelpy LLM Gemini Pro and Kelper LLM Claude 3 Haiku! It's great to meet both of you as well. I'm excited to work together in managing our kelp bed and growing some healthy kelp. Let's make this a successful venture!
------------------------------------------------------------


In [13]:
print(httpx.get("https://www.ndbc.noaa.gov/data/latest_obs/46092.txt").text)

Station 46092
36° 45.1' N  122° 1.8' W

1:14 pm PDT
2014 GMT 05/04/24
Wind: WNW (290°), 13.6 kt
Pres: 29.89 rising
Air Temp: 49.1 °F
Water Temp: 50.9 °F



In [39]:
import re
from datetime import datetime

def parse_buoy_data(data: str) -> dict:
    # Regular expressions for each field
    station_id_pattern = r"Station (\d+)"
    latitude_pattern = r"(\d+)° (\d+\.\d+)' (\w)"
    longitude_pattern = r"(\d+)° (\d+\.\d+)' (\w)"
    time_pattern = r"(\d+):(\d+) (\w+) (\w+)"
    date_pattern = r"(\d+)/(\d+)/(\d+)"
    wind_pattern = r"Wind: (\w+) \((\d+)°\), (\d+\.\d+) kt"
    pressure_pattern = r"Pres: (\d+\.\d+) (\w+)"
    air_temp_pattern = r"Air Temp: (\d+\.\d+) °F"
    water_temp_pattern = r"Water Temp: (\d+\.\d+) °F"

    output = {
        "station_id": None,
        "latitude": None,
        "longitude": None,
        "date": None,
        "wind_direction": None,
        "wind_direction_deg": None,
        "wind_speed": None,
        "pressure": None,
        "pressure_tendency": None,
        "air_temp_c": None,
        "water_temp_c": None
        }
    
    # Match each field using the corresponding pattern
    station_id_match = re.search(station_id_pattern, data)
    if station_id_match:
        output["station_id"] = int(station_id_match.group(1))
    latitude_match = re.search(latitude_pattern, data)
    if latitude_match:
        latitude_degrees = float(latitude_match.group(1))
        latitude_minutes = float(latitude_match.group(2))
        latitude_dir = latitude_match.group(3)
        latitude = latitude_degrees + latitude_minutes / 60
        if latitude_dir.upper() == "S":
            latitude = -latitude
        output["latitude"] = latitude
    longitude_match = re.search(longitude_pattern, data)
    if longitude_match:
        longitude_degrees = float(longitude_match.group(1))
        longitude_minutes = float(longitude_match.group(2))
        longitude_dir = longitude_match.group(3)
        longitude = longitude_degrees + longitude_minutes / 60
        if longitude_dir.upper() == "W":
            longitude = -longitude
        output["longitude"] = longitude
        
    time_match = re.search(time_pattern, data)
    date_match = re.search(date_pattern, data)
    if date_match:
        month = int(date_match.group(1))
        day = int(date_match.group(2))
        year = int(date_match.group(3))
        timestamp = datetime(year, month, day)
        if time_match:
            hour = int(time_match.group(1))
            minute = int(time_match.group(2))
            timestamp = datetime(year, month, day, hour, minute)
        output["date"] = timestamp.isoformat()
    wind_match = re.search(wind_pattern, data)
    if wind_match:
        output["wind_direction"] = wind_match.group(1)
        output["wind_direction_deg"] = int(wind_match.group(2))
        output["wind_speed"] = float(wind_match.group(3))
    pressure_match = re.search(pressure_pattern, data)
    if pressure_match:
        output["pressure"] = float(pressure_match.group(1))
        output["pressure_tendency"] = pressure_match.group(2)
    air_temp_match = re.search(air_temp_pattern, data)
    if air_temp_match:
        air_temp_f = float(air_temp_match.group(1))
        output["air_temp_c"] = (air_temp_f - 32) * 5 / 9
    water_temp_match = re.search(water_temp_pattern, data)
    if water_temp_match:
        water_temp_f = float(water_temp_match.group(1))
        output["water_temp_c"] = (water_temp_f - 32) * 5 / 9
    return output

In [40]:
buoy_data = parse_buoy_data(httpx.get("https://www.ndbc.noaa.gov/data/latest_obs/46092.txt").text)

In [41]:
buoy_data

{'station_id': 46092,
 'latitude': 36.751666666666665,
 'longitude': 36.751666666666665,
 'date': '0024-05-05T09:14:00',
 'wind_direction': 'NW',
 'wind_direction_deg': 320,
 'wind_speed': 3.9,
 'pressure': 30.09,
 'pressure_tendency': 'rising',
 'air_temp_c': 10.222222222222221,
 'water_temp_c': 10.111111111111112}